<a href="https://colab.research.google.com/github/ibabedal/BLCA-analysis/blob/main/Bladder_Cancer_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Data

In [1]:
!wget https://cbioportal-datahub.s3.amazonaws.com/blca_tcga_pan_can_atlas_2018.tar.gz
!ls -l 

--2022-10-07 06:48:52--  https://cbioportal-datahub.s3.amazonaws.com/blca_tcga_pan_can_atlas_2018.tar.gz
Resolving cbioportal-datahub.s3.amazonaws.com (cbioportal-datahub.s3.amazonaws.com)... 52.216.138.203
Connecting to cbioportal-datahub.s3.amazonaws.com (cbioportal-datahub.s3.amazonaws.com)|52.216.138.203|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 207774420 (198M) [application/x-tar]
Saving to: ‘blca_tcga_pan_can_atlas_2018.tar.gz’

blca_tcga_pan_can_a 100%[===================>] 198.15M  34.6MB/s    in 5.8s    

2022-10-07 06:48:58 (34.0 MB/s) - ‘blca_tcga_pan_can_atlas_2018.tar.gz’ saved [207774420/207774420]

total 202912
-rw-r--r-- 1 root root 207774420 Aug 13 02:01 blca_tcga_pan_can_atlas_2018.tar.gz
drwxr-xr-x 1 root root      4096 Oct  5 13:35 sample_data


In [3]:
!tar zxf blca_tcga_pan_can_atlas_2018.tar.gz

In [5]:
!cp ./blca_tcga_pan_can_atlas_2018/data_cna.txt ./blca_tcga_pan_can_atlas_2018/data_clinical_sample.txt ./blca_tcga_pan_can_atlas_2018/data_mrna_seq_v2_rsem.txt \
./blca_tcga_pan_can_atlas_2018/data_methylation_hm27_hm450_merged.txt  .
!ls -l 

total 450932
drwxr-xr-x 4 1001  121      4096 Aug 13 02:00 blca_tcga_pan_can_atlas_2018
-rw-r--r-- 1 root root 207774420 Aug 13 02:01 blca_tcga_pan_can_atlas_2018.tar.gz
-rw-r--r-- 1 root root     92838 Oct  7 06:53 data_clinical_sample.txt
-rw-r--r-- 1 root root  23352498 Oct  7 06:53 data_cna.txt
-rw-r--r-- 1 root root 172689278 Oct  7 06:53 data_methylation_hm27_hm450_merged.txt
-rw-r--r-- 1 root root  57824648 Oct  7 06:53 data_mrna_seq_v2_rsem.txt
drwxr-xr-x 1 root root      4096 Oct  5 13:35 sample_data


In [6]:
# skiping the first 4 lines in clinical data as tehy are not acutal samples but information about them
!tail +5 data_clinical_sample.txt > data_clinical_sample_updated.txt

In [7]:
!head data_clinical_sample_updated.txt

PATIENT_ID	SAMPLE_ID	ONCOTREE_CODE	CANCER_TYPE	CANCER_TYPE_DETAILED	TUMOR_TYPE	GRADE	TISSUE_PROSPECTIVE_COLLECTION_INDICATOR	TISSUE_RETROSPECTIVE_COLLECTION_INDICATOR	TISSUE_SOURCE_SITE_CODE	TUMOR_TISSUE_SITE	ANEUPLOIDY_SCORE	SAMPLE_TYPE	MSI_SCORE_MANTIS	MSI_SENSOR_SCORE	SOMATIC_STATUS	TMB_NONSYNONYMOUS	TISSUE_SOURCE_SITE
TCGA-2F-A9KO	TCGA-2F-A9KO-01	BLCA	Bladder Cancer	Bladder Urothelial Carcinoma	Muscle Invasive Urothelial Carcinoma (PT2 or Above)	High Grade	No	Yes	2F	Bladder	19	Primary	0.3271	0	Matched	16.66666667	Erasmus MC
TCGA-2F-A9KP	TCGA-2F-A9KP-01	BLCA	Bladder Cancer	Bladder Urothelial Carcinoma	Muscle Invasive Urothelial Carcinoma (PT2 or Above)	High Grade	No	Yes	2F	Bladder	10	Primary	0.3234	0.17	Matched	4.4	Erasmus MC
TCGA-2F-A9KQ	TCGA-2F-A9KQ-01	BLCA	Bladder Cancer	Bladder Urothelial Carcinoma	Muscle Invasive Urothelial Carcinoma (PT2 or Above)	High Grade	No	Yes	2F	Bladder	6	Primary	0.3181	0.12	Matched	2.066666667	Erasmus MC
TCGA-2F-A9KR	TCGA-2F-A9KR-01	BLCA	Bladder Cancer	

# Loading Libraries

In [8]:
import pandas as pd
import numpy as np

#Clinical Data

In [9]:
df_clinical = pd.read_csv('./data_clinical_sample_updated.txt', sep='\t')
df_clinical.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 18 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   PATIENT_ID                                 411 non-null    object 
 1   SAMPLE_ID                                  411 non-null    object 
 2   ONCOTREE_CODE                              411 non-null    object 
 3   CANCER_TYPE                                411 non-null    object 
 4   CANCER_TYPE_DETAILED                       411 non-null    object 
 5   TUMOR_TYPE                                 411 non-null    object 
 6   GRADE                                      408 non-null    object 
 7   TISSUE_PROSPECTIVE_COLLECTION_INDICATOR    411 non-null    object 
 8   TISSUE_RETROSPECTIVE_COLLECTION_INDICATOR  410 non-null    object 
 9   TISSUE_SOURCE_SITE_CODE                    411 non-null    object 
 10  TUMOR_TISSUE_SITE         

In [10]:
df_clinical_updated = df_clinical[['SAMPLE_ID', 'TMB_NONSYNONYMOUS']]
df_clinical_updated.head()

,SAMPLE_ID,TMB_NONSYNONYMOUS
0,TCGA-2F-A9KO-01,16.666667
1,TCGA-2F-A9KP-01,4.400000
2,TCGA-2F-A9KQ-01,2.066667
3,TCGA-2F-A9KR-01,7.000000
4,TCGA-2F-A9KT-01,11.300000


In [11]:
tmb_median = df_clinical_updated['TMB_NONSYNONYMOUS'].median()
print(tmb_median)

5.766666667


In [12]:
df_clinical_updated['TMB_CLASS'] = np.where(df_clinical_updated['TMB_NONSYNONYMOUS'] < tmb_median, 1, 2)
df_clinical_updated.head()
df_clinical_updated.to_csv('./data_clinical_updated.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
df_clinical_to_merge = df_clinical_updated[['SAMPLE_ID','TMB_CLASS']]

# CNA

In [14]:
df_cna = pd.read_csv('./data_cna.txt',sep='\t')
df_cna.shape

(25128, 410)

In [15]:
df_cna.drop(columns='Entrez_Gene_Id',inplace=True)
df_cna.head()

,Hugo_Symbol,TCGA-2F-A9KO-01,TCGA-2F-A9KP-01,TCGA-2F-A9KQ-01,TCGA-2F-A9KR-01,TCGA-2F-A9KT-01,TCGA-2F-A9KW-01,TCGA-4Z-AA7M-01,TCGA-4Z-AA7N-01,TCGA-4Z-AA7O-01,...,TCGA-ZF-AA4X-01,TCGA-ZF-AA51-01,TCGA-ZF-AA52-01,TCGA-ZF-AA53-01,TCGA-ZF-AA54-01,TCGA-ZF-AA56-01,TCGA-ZF-AA58-01,TCGA-ZF-AA5H-01,TCGA-ZF-AA5N-01,TCGA-ZF-AA5P-01
0,ACAP3,-1,2,0,0,0,-2,0,0,0,...,-1,1,1,1,-1,0,0,1,0,0
1,ACTRT2,-1,2,0,0,0,-2,0,0,0,...,-1,1,1,1,-1,0,0,1,0,0
2,AGRN,-1,2,0,0,0,-2,0,0,0,...,-1,1,1,1,-1,0,0,1,0,0
3,ANKRD65,-1,2,0,0,0,-2,0,0,0,...,-1,1,1,1,-1,0,0,1,0,0
4,ATAD3A,-1,2,0,0,0,-2,0,0,0,...,-1,1,1,1,-1,0,0,1,0,0


In [16]:
df_cna.isnull().sum().sum()

0

In [17]:
df_cna.fillna(0,inplace=True)

In [18]:
df_cna_corrected = df_cna.transpose(copy=True)

In [19]:
df_cna_corrected = df_cna_corrected.reset_index(drop=False)
df_cna_corrected['index'] = df_cna_corrected['index'].str.replace('Hugo_Symbol','SAMPLE_ID')
df_cna_corrected.columns = df_cna_corrected.iloc[0]
df_cna_corrected.drop(df_cna_corrected.index[0],inplace=True)
df_cna_corrected = df_cna_corrected.reset_index(drop=True)
df_cna_corrected.head()

,SAMPLE_ID,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A,ATAD3B,ATAD3C,AURKAIP1,B3GALT6,...,hsa-mir-1321,hsa-mir-361,hsa-mir-548m,hsa-mir-652,hsa-mir-220a,hsa-mir-513c,hsa-mir-513b,hsa-mir-513a-1,hsa-mir-513a-2,hsa-mir-224
0,TCGA-2F-A9KO-01,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
1,TCGA-2F-A9KP-01,2,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
2,TCGA-2F-A9KQ-01,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,TCGA-2F-A9KR-01,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,0,0,0,0,-1
4,TCGA-2F-A9KT-01,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [20]:
#list_of_nulls = df_cna_corrected.columns[df_cna_corrected.isna().any()].tolist()
#df_cna_corrected[list_of_nulls][df_cna_corrected.isnull().any(axis=1)]

In [21]:
#float(df_cna_corrected['ADAM3A'].mode())

In [22]:
df_cna_corrected.isnull().sum().sum()

0

In [23]:
df_cna_corrected.to_csv('./data_cna_updated.csv',index=False)

#### merging cna with tmb_class

In [24]:
df_cna_tmb = pd.merge(df_cna_corrected, df_clinical_to_merge, on='SAMPLE_ID', how='inner')
df_cna_tmb.head()

,SAMPLE_ID,ACAP3,ACTRT2,AGRN,ANKRD65,ATAD3A,ATAD3B,ATAD3C,AURKAIP1,B3GALT6,...,hsa-mir-361,hsa-mir-548m,hsa-mir-652,hsa-mir-220a,hsa-mir-513c,hsa-mir-513b,hsa-mir-513a-1,hsa-mir-513a-2,hsa-mir-224,TMB_CLASS
0,TCGA-2F-A9KO-01,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,0,0,0,0,0,0,0,0,0,2
1,TCGA-2F-A9KP-01,2,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,1
2,TCGA-2F-A9KQ-01,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
3,TCGA-2F-A9KR-01,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,0,0,0,0,-1,2
4,TCGA-2F-A9KT-01,0,0,0,0,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,2


In [25]:
df_cna_tmb.to_csv('./CNA-TMB.csv',index=False)

# mRNA

In [44]:
df_mrna = pd.read_csv('./data_mrna_seq_v2_rsem.txt',sep='\t')
df_mrna.shape

(20531, 409)

In [45]:

df_mrna.head()

,Hugo_Symbol,Entrez_Gene_Id,TCGA-2F-A9KO-01,TCGA-2F-A9KP-01,TCGA-2F-A9KQ-01,TCGA-2F-A9KR-01,TCGA-2F-A9KT-01,TCGA-2F-A9KW-01,TCGA-4Z-AA7M-01,TCGA-4Z-AA7N-01,...,TCGA-ZF-AA4X-01,TCGA-ZF-AA51-01,TCGA-ZF-AA52-01,TCGA-ZF-AA53-01,TCGA-ZF-AA54-01,TCGA-ZF-AA56-01,TCGA-ZF-AA58-01,TCGA-ZF-AA5H-01,TCGA-ZF-AA5N-01,TCGA-ZF-AA5P-01
0,NaN,100130426,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.4411,0.4216,0.0000,1.227,0.0000
1,NaN,100133144,20.4373,16.1382,13.3333,15.3523,14.0136,7.1159,22.4891,26.1199,...,24.0360,0.0000,4.2992,0.0000,4.6306,5.8271,1.7032,3.3904,0.000,7.0317
2,UBE2Q2P2,100134869,37.8717,12.5759,10.7742,42.5810,17.6427,2.3091,45.0490,25.2546,...,10.3812,5.2751,14.5465,6.8871,8.0213,7.8474,2.0911,1.1387,0.000,4.0856
3,HMGB1P1,10357,123.0900,137.8860,104.6780,146.4530,142.9620,176.8770,126.3560,97.3321,...,58.4972,119.0880,91.4959,163.8500,83.2794,185.6730,210.1810,186.9380,155.301,114.1130
4,NaN,10431,702.0410,882.2310,954.1030,487.2100,954.7770,1586.0300,634.5320,642.6320,...,841.2210,859.0810,753.8280,1646.6900,1702.4300,575.2100,1595.7000,1100.5700,1123.930,778.2100


In [46]:
df_mrna.isnull().sum().sum()

13

In [47]:
df_mrna.columns[df_mrna.isna().any()].tolist()

['Hugo_Symbol']

In [51]:
list_of_nulls = df_mrna.columns[df_mrna.isna().any()].tolist()
df_mrna[df_mrna.isnull().any(axis=1)]

,Hugo_Symbol,Entrez_Gene_Id,TCGA-2F-A9KO-01,TCGA-2F-A9KP-01,TCGA-2F-A9KQ-01,TCGA-2F-A9KR-01,TCGA-2F-A9KT-01,TCGA-2F-A9KW-01,TCGA-4Z-AA7M-01,TCGA-4Z-AA7N-01,...,TCGA-ZF-AA4X-01,TCGA-ZF-AA51-01,TCGA-ZF-AA52-01,TCGA-ZF-AA53-01,TCGA-ZF-AA54-01,TCGA-ZF-AA56-01,TCGA-ZF-AA58-01,TCGA-ZF-AA5H-01,TCGA-ZF-AA5N-01,TCGA-ZF-AA5P-01
0,NaN,100130426,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.4411,0.4216,0.0000,1.2270,0.0000
1,NaN,100133144,20.4373,16.1382,13.3333,15.3523,14.0136,7.1159,22.4891,26.1199,...,24.0360,0.0000,4.2992,0.0000,4.6306,5.8271,1.7032,3.3904,0.0000,7.0317
4,NaN,10431,702.0410,882.2310,954.1030,487.2100,954.7770,1586.0300,634.5320,642.6320,...,841.2210,859.0810,753.8280,1646.6900,1702.4300,575.2100,1595.7000,1100.5700,1123.9300,778.2100
5,NaN,136542,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
6,NaN,155060,406.9970,182.2720,224.8490,472.9640,234.5960,75.4006,520.1530,292.9250,...,1213.0100,157.4980,325.6770,64.0496,114.3730,88.6634,193.0860,136.1960,233.1290,221.2340
9,NaN,317712,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
13,NaN,391343,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0893,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
18,NaN,553137,7.5802,0.8323,0.0000,0.0000,1.1306,23.0391,3.2680,0.0000,...,2.0010,0.7536,12.9564,6.1983,0.0000,0.0000,0.0000,2.2645,0.0000,2.2235
19,NaN,57714,3076.3800,266.7500,307.3710,853.8050,227.8120,247.1460,305.5560,892.2940,...,660.7300,385.0790,478.7990,181.8180,406.3770,609.6160,526.1380,954.3450,181.5950,198.9990
20,NaN,645851,23.3236,88.2231,12.0538,28.0170,32.2216,10.9959,69.1721,23.4340,...,17.6088,9.0430,1.7668,6.1983,8.0972,13.6745,3.7943,6.7936,7.3620,20.5670


In [80]:
# dropping the missing na as recover is not possible
df_mrna.dropna(inplace=True)

#### trying to recover the missing gene names from the file data_mrna_seq_v2_rsem_zscores_ref_diploid_samples.txt as it has all the names

In [73]:
## trying to get the missing gene names from the file data_mrna_seq_v2_rsem_zscores_ref_diploid_samples.txt as it has all the names
df_mrna_temp = pd.read_csv('./blca_tcga_pan_can_atlas_2018/data_mrna_seq_v2_rsem_zscores_ref_diploid_samples.txt',sep='\t')

In [74]:
df_mrna_temp.isnull().sum().sum()

219780

In [75]:
df_mrna_temp.shape

(20471, 409)

In [ ]:
# most of the nulls are in the values for the samples not on the gene names, we will use the coloumn Entrez_Gene_Id for mapping
df_mrna_temp.columns[df_mrna_temp.isna().any()].tolist()

In [77]:
# getting the values mapping to Entrez_Gene_Id NaN values
entrez_gene_id = df_mrna['Entrez_Gene_Id'][df_mrna.isnull().any(axis=1)]
list(entrez_gene_id)

[100130426,
 100133144,
 10431,
 136542,
 155060,
 317712,
 391343,
 553137,
 57714,
 645851,
 652919,
 728603,
 729884]

In [79]:
for i in list(entrez_gene_id):
  try:
    hugo_symbol = df_mrna_temp.loc[df_mrna_temp['Entrez_Gene_Id'] == i, 'Hugo_Symbol'].item()
    print(hugo_symbol, i)
    
  except:
    pass

LOC155060 155060
LOC391343 391343
LOC553137 553137


#### continue normal work

In [81]:
df_mrna.drop(columns='Entrez_Gene_Id',inplace=True)
df_mrna_corrected = df_mrna.transpose(copy=True)

In [82]:
df_mrna_corrected = df_mrna_corrected.reset_index(drop=False)
df_mrna_corrected['index'] = df_mrna_corrected['index'].str.replace('Hugo_Symbol','SAMPLE_ID')
df_mrna_corrected.columns = df_mrna_corrected.iloc[0]
df_mrna_corrected.drop(df_mrna_corrected.index[0],inplace=True)
df_mrna_corrected = df_mrna_corrected.reset_index(drop=True)
df_mrna_corrected.head()

,SAMPLE_ID,UBE2Q2P2,HMGB1P1,RNU12-2P,SSX9P,EZHIP,EFCAB8,SRP14P1,TRIM75P,SPATA31B1P,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,TCGA-2F-A9KO-01,37.8717,123.09,0.5831,0.0,11.0787,2.3324,4.0816,0.0,0.0,...,694.857,554.519,102.624,776.676,1450.73,5.8309,697.959,4262.39,1787.76,572.595
1,TCGA-2F-A9KP-01,12.5759,137.886,0.8323,0.0,0.0,4.5776,2.0807,0.4161,0.0,...,718.119,969.621,37.037,316.688,1420.72,17.062,1526.43,3105.29,1467.75,860.591
2,TCGA-2F-A9KQ-01,10.7742,104.678,0.0,0.0,0.0,0.4636,2.318,0.0,0.0,...,532.536,795.086,60.2689,378.303,873.435,1.8544,1571.16,3275.38,750.58,621.233
3,TCGA-2F-A9KR-01,42.581,146.453,2.3743,0.0,1.8995,1.4246,3.7989,0.0,0.0,...,616.848,568.412,45.5869,470.115,1924.15,1.8995,748.86,4465.62,910.788,935.481
4,TCGA-2F-A9KT-01,17.6427,142.962,0.0,0.0,5.0876,2.2612,6.2182,0.0,0.0,...,726.625,1087.05,17.524,281.515,938.383,6.7835,878.462,3449.41,850.198,630.865


In [83]:
df_mrna_corrected.isnull().sum().sum()

0

In [84]:
list_of_nulls = df_mrna_corrected.columns[df_mrna_corrected.isna().any()].tolist()
df_mrna_corrected[list_of_nulls][df_mrna_corrected.isnull().any(axis=1)]

""


In [ ]:
##list_of_nulls = df_mrna_corrected.columns[df_mrna_corrected.isna().any()].tolist()
##for i in list_of_nulls:
  ##mean_value = df_mrna_corrected[i].mean()
  #print(mean_value)
  #print(df_mrna_corrected[i].head(3))
  ##df_mrna_corrected[i] = df_mrna_corrected[i].fillna(value=mean_value, axis=0)
  #print(df_mrna_corrected[i][df_mrna_corrected.isnull().any(axis=1)])

In [85]:
#df_mrna_corrected[list_of_nulls][df_mrna_corrected.isnull().any(axis=1)]

In [86]:
df_mrna_corrected.isnull().sum().sum()

0

In [87]:
df_mrna_corrected.to_csv('./data_mrna_updated.csv', index=False)

## merging the mRNA with TMB

In [88]:
df_mrna_tmb = pd.merge(df_mrna_corrected, df_clinical_to_merge, on='SAMPLE_ID', how='inner')
df_mrna_tmb.head()

,SAMPLE_ID,UBE2Q2P2,HMGB1P1,RNU12-2P,SSX9P,EZHIP,EFCAB8,SRP14P1,TRIM75P,SPATA31B1P,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,TMB_CLASS
0,TCGA-2F-A9KO-01,37.8717,123.09,0.5831,0.0,11.0787,2.3324,4.0816,0.0,0.0,...,554.519,102.624,776.676,1450.73,5.8309,697.959,4262.39,1787.76,572.595,2
1,TCGA-2F-A9KP-01,12.5759,137.886,0.8323,0.0,0.0,4.5776,2.0807,0.4161,0.0,...,969.621,37.037,316.688,1420.72,17.062,1526.43,3105.29,1467.75,860.591,1
2,TCGA-2F-A9KQ-01,10.7742,104.678,0.0,0.0,0.0,0.4636,2.318,0.0,0.0,...,795.086,60.2689,378.303,873.435,1.8544,1571.16,3275.38,750.58,621.233,1
3,TCGA-2F-A9KR-01,42.581,146.453,2.3743,0.0,1.8995,1.4246,3.7989,0.0,0.0,...,568.412,45.5869,470.115,1924.15,1.8995,748.86,4465.62,910.788,935.481,2
4,TCGA-2F-A9KT-01,17.6427,142.962,0.0,0.0,5.0876,2.2612,6.2182,0.0,0.0,...,1087.05,17.524,281.515,938.383,6.7835,878.462,3449.41,850.198,630.865,2


In [89]:
df_mrna_tmb.to_csv('./mRNA-TMB.csv',index=False)

# Starting with DNA

In [91]:
df_dna = pd.read_csv('./data_methylation_hm27_hm450_merged.txt', sep='\t')

In [92]:
df_dna.shape

(22601, 415)

In [96]:
df_dna.iloc[:, :5].head(7)

,ENTITY_STABLE_ID,NAME,DESCRIPTION,TRANSCRIPT_ID,TCGA-2F-A9KO-01
0,cg00000292,ATP2A1,1stExon,NM_173201;NM_004320,0.680818
1,cg00003994,MEOX2,1stExon,NM_005924,0.062263
2,cg00005847,HOXD3,5'UTR,NM_006898,0.496348
3,cg00007981,PANX1,1stExon,NM_015368,0.028165
4,cg00008493,KIAA1409;COX8C,Body;5'UTR,NM_020818;NM_182971,0.902485
5,cg00008713,IMPA2,TSS1500,NM_014214,0.049487
6,cg00009407,TTC8,TSS200,NM_144596;NM_198310;NM_198309,0.030269
